In [11]:
import os
import re
import random
import copy
from collections import Counter
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gc
    
from gensim.models import KeyedVectors

import numpy as np
from numpy.random import seed
seed(1000)
from tensorflow.random import set_random_seed
set_random_seed(1000)

import pandas
from pandas import Series, DataFrame

import keras
import keras.backend as K
# from keras.layers import Layer
from keras.engine import Layer
from keras.engine import InputSpec
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Embedding, Conv1D, Concatenate, Flatten, Dropout, Dense, Bidirectional, Activation
from keras.layers import MaxPooling1D, Conv2D, Flatten, Reshape, MaxPooling2D, LSTM, Lambda, Highway, Dot
from keras.models import Model, Sequential
from keras.activations import softmax
from keras.initializers import Constant, RandomNormal, RandomUniform

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

import matplotlib.pyplot as plt
import seaborn as sns       

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

#进行配置，使用30%的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

In [16]:
# 文件路径
ORIGINAL_FILE_PATH = "../data/original_data/quora_duplicate_questions.tsv"

DATA_ALL_PATH = "../data/all_data.txt"

DATA_TRAIN_PATH = "../data/train.txt"
DATA_VALID_PATH = "../data/valid.txt"
DATA_TEST_PATH = "../data/test.txt"

MAPS_FILE_PATH = "../data/maps.pkl"

# Glove文件
WORD2VEC_PATH = "../data/GoogleNews-vectors-negative300.bin"

# Model Path
MODEL_PATH = "../model/conv_knrm/conv_knrm-01.h5"

# 创建文件夹
(file_path, file_name) = os.path.split(MODEL_PATH)
if not os.path.exists(file_path):
    os.mkdir(file_path)

In [3]:
def load_dict(dict_path):
    with open(dict_path, 'rb') as fr:
        return pickle.load(fr)
        
def create_maps(dico):
    item2id = dico
    id2item = dict([val, key] for key, val in dico.items())
    return item2id, id2item

def read_file(fpath):
    """
    读取文件，返回
    """
    question1, question2, labels = [], [], []
    with open(fpath, 'r', encoding="utf-8") as fr:
        for line in fr.readlines():
            try:
                q, k, l = line.strip().split('\t')
            except ValueError:
                print(line)
            question1.append(q)
            question2.append(k)
            labels.append(int(l))
            
    return question1, question2, labels

def load_data(fpath, word_to_id, max_length=20):
    """
    载入数据，并将其转化为id表示
    """
    question1, question2, labels = read_file(fpath)
    
    q1_id, q2_id = [], []
    for i in range(len(question1)):
        q1= question1[i].split()
        q2 = question2[i].split()
        q1_id.append([word_to_id[x] for x in q1 if x in word_to_id])
        q2_id.append([word_to_id[x] for x in q2 if x in word_to_id])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x1_pad = keras.preprocessing.sequence.pad_sequences(q1_id, max_length, padding="post")
    x2_pad = keras.preprocessing.sequence.pad_sequences(q2_id, max_length, padding="post")
    labels = keras.utils.np_utils.to_categorical(labels)
    
    return x1_pad, x2_pad, labels
    
        
def load_word2vec(embedding_path, word_index, embed_dim=300): 
    """
    载入预训练好的word2vec(Google-News)
    """
    # 载入word2vec词向量
    word2vec_dict = KeyedVectors.load_word2vec_format(embedding_path,binary=True)
    
    embedding_index = dict()
    for word in word2vec_dict.wv.vocab:
        embedding_index[word] = word2vec_dict.word_vec(word)
    print('Load %s word vectors.' % len(embedding_index))
    
    all_embs = np.stack(list(embedding_index.values()))
    # emb_mean, emb_std = all_embs.mean(), all_embs.std()
    emb_mean = np.mean(all_embs, axis=0)
    vocab_size = len(word_index)
    # 初始化权重
    embedding_matrix = np.zeros((vocab_size+1, embed_dim))
    gc.collect()
    # 对权重矩阵进行赋值，未找到时用词向量平均值填充
    for word, i in word_index.items():
        if i < vocab_size + 1:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = emb_mean
    del embedding_index
    return embedding_matrix

In [4]:
dico = load_dict(MAPS_FILE_PATH)
word_index, index_word = create_maps(dico)

# 准备数据
x1_train, x2_train, y_train = load_data(DATA_TRAIN_PATH, word_index)
x1_valid, x2_valid, y_valid = load_data(DATA_VALID_PATH, word_index)
x1_test, x2_test, y_test = load_data(DATA_TEST_PATH, word_index)

embedding_matrix = load_word2vec(WORD2VEC_PATH, word_index)

Load 3000000 word vectors.


In [9]:
vocab_size = len(dico)

class Settings(object):
    text1_maxlen = 20
    text2_maxlen = 20
    embedding_dim = 300
    vocab_size = vocab_size

    max_ngram = 3
    num_filters = 64
    kernel_num = 64
    sigma = 0.1
    exact_sigma = 0.001
    if_crossmatch = True
    
    dropout_rate = 0.2
    
settings = Settings()

In [15]:
# 定义网络
def kernel_layer(mu, sigma):
    def kernel(x):
        return K.tf.exp(-0.5 * (x - mu)*(x-mu)/sigma/sigma)
    return Activation(kernel)

query = Input(name='query', shape=(settings.text1_maxlen,))
doc = Input(name='doc', shape=(settings.text2_maxlen,))


embedding = Embedding(settings.vocab_size + 1,
                      settings.embedding_dim,
                      weights=[embedding_matrix],
                      input_length=settings.text1_maxlen,
                      trainable=False)

q_embed = embedding(query)
d_embed = embedding(doc)

convs = []
q_convs = []
d_convs = []
for i in range(settings.max_ngram):
    c = Conv1D(settings.num_filters, i+1, activation="relu", padding="same")
    q_convs.append(c(q_embed))
    d_convs.append(c(d_embed))
    
KM = []

for qi in range(settings.max_ngram):
    for di in range(settings.max_ngram):
        if not settings.if_crossmatch and qi != di:
            print("non cross")
            continue
        q_ngram = q_convs[qi]
        d_ngram = d_convs[di]
        mm = Dot(axes=[2, 2], normalize=True)([q_ngram, d_ngram])
        for i in range(settings.kernel_num):
            mu = 1. / (settings.kernel_num - 1) + (2. * i) /(settings.kernel_num - 1) - 1.0
            sigma = settings.sigma
            if mu > 1.0:
                sigma = settings.exact_sigma
                mu = 1.0
            mm_exp = kernel_layer(mu, sigma)(mm)
            mm_doc_sum = Lambda(lambda x: K.tf.reduce_sum(x,2))(mm_exp)
            mm_log = Activation(K.tf.log1p)(mm_doc_sum)
            mm_sum = Lambda(lambda x: K.tf.reduce_sum(x, 1))(mm_log)
            KM.append(mm_sum)
            
Phi = Lambda(lambda x: K.tf.stack(x, 1))(KM)

out_ = Dense(2, activation='softmax', kernel_initializer=RandomUniform(minval=-0.014, maxval=0.014), bias_initializer='zeros')(Phi)

model = Model(inputs=[query, doc], outputs=out_)
model.summary()        

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 20)           0                                            
__________________________________________________________________________________________________
doc (InputLayer)                (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 20, 300)      25293600    query[0][0]                      
                                                                 doc[0][0]                        
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 20, 64)       19264       embedding_4[0][0]                
          

__________________________________________________________________________________________________
lambda_511 (Lambda)             (None, 20)           0           activation_512[0][0]             
__________________________________________________________________________________________________
lambda_513 (Lambda)             (None, 20)           0           activation_514[0][0]             
__________________________________________________________________________________________________
lambda_515 (Lambda)             (None, 20)           0           activation_516[0][0]             
__________________________________________________________________________________________________
lambda_517 (Lambda)             (None, 20)           0           activation_518[0][0]             
__________________________________________________________________________________________________
lambda_519 (Lambda)             (None, 20)           0           activation_520[0][0]             
__________

activation_859 (Activation)     (None, 20)           0           lambda_857[0][0]                 
__________________________________________________________________________________________________
activation_861 (Activation)     (None, 20)           0           lambda_859[0][0]                 
__________________________________________________________________________________________________
activation_863 (Activation)     (None, 20)           0           lambda_861[0][0]                 
__________________________________________________________________________________________________
activation_865 (Activation)     (None, 20)           0           lambda_863[0][0]                 
__________________________________________________________________________________________________
activation_867 (Activation)     (None, 20)           0           lambda_865[0][0]                 
__________________________________________________________________________________________________
activation

                                                                 lambda_1106[0][0]                
                                                                 lambda_1108[0][0]                
                                                                 lambda_1110[0][0]                
                                                                 lambda_1112[0][0]                
                                                                 lambda_1114[0][0]                
                                                                 lambda_1116[0][0]                
                                                                 lambda_1118[0][0]                
                                                                 lambda_1120[0][0]                
                                                                 lambda_1122[0][0]                
                                                                 lambda_1124[0][0]                
          

In [17]:
import datetime,time
from keras.optimizers import Adam

callback_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor='val_acc',
        save_best_only=True
    )
]

adadelta=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['binary_accuracy'])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

print('Train...')

model.fit([x1_train, x2_train], y_train, epochs=100, batch_size=128, callbacks=callback_list, validation_data=([x1_valid, x2_valid], y_valid))

Train...
Train on 323426 samples, validate on 40428 samples
Epoch 1/100
323426/323426 [==============================] - 838s 3ms/step - loss: 0.5447 - acc: 0.7322 - val_loss: 0.4252 - val_acc: 0.7959
Epoch 2/100
323426/323426 [==============================] - 802s 2ms/step - loss: 0.4592 - acc: 0.7833 - val_loss: 0.4072 - val_acc: 0.8116
Epoch 3/100
323426/323426 [==============================] - 806s 2ms/step - loss: 0.4305 - acc: 0.8019 - val_loss: 0.4269 - val_acc: 0.8062
Epoch 4/100
323426/323426 [==============================] - 805s 2ms/step - loss: 0.3995 - acc: 0.8182 - val_loss: 0.4099 - val_acc: 0.8169
Epoch 5/100
323426/323426 [==============================] - 805s 2ms/step - loss: 0.3841 - acc: 0.8284 - val_loss: 0.4944 - val_acc: 0.7821
Epoch 6/100
323426/323426 [==============================] - 805s 2ms/step - loss: 0.3642 - acc: 0.8394 - val_loss: 0.4077 - val_acc: 0.8233
Epoch 7/100
323426/323426 [==============================] - 801s 2ms/step - loss: 0.3572 - ac

In [22]:
y_vec_pred = model.predict([x1_test, x2_test], batch_size=128)

In [23]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

def to_one_hot(prob):
    prob_shape = prob.shape
    y_pred = np.zeros(prob_shape)
    
    i = 0
    for item in prob:
        if item[0] < item[1]:
            y_pred[i] = np.array([0, 1])
        else:
            y_pred[i] = np.array([1, 0])
        i += 1
    return y_pred

y_pred = to_one_hot(y_vec_pred)
print(y_pred)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [24]:
report = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
def print_prf(result):
    precision = str(round(result[0] * 100, 2)) + "%"
    recall = str(round(result[1] * 100, 2)) + "%"
    f1 = str(round(result[2] * 100, 2)) + "%"
    
    strResult="Precision:"+precision+"\tRecall:"+recall+"\tF-score:"+f1
    print(strResult)
    
print_prf(report)

Precision:79.93%	Recall:80.25%	F-score:80.08%


In [25]:
model.evaluate([x1_test, x2_test], y_test)

40429/40429 [==============================] - 70s 2ms/step


[1.4226651384583588, 0.8129807811268845]